In [ ]:
#| echo: false
using DataFrames, Plots, Random, Distributions
using EcologicalNetworks, EcologicalNetworksPlots, BEFWM2

Navigate to the [GitHub location for the BioEnergetic Model](https://github.com/BecksLab/EcologicalNetworksDynamics.jl).

It should look like this:

![GitHubPageForBEFW](Figs/BEFWGit.png)

Then download to your Documents Folder the package.

![DownloadTheZip](Figs/DownloadZip.png)

Once this folder is unzipped in your Documents Folder, you can _install_ it using `Pkg.develop`.


In [ ]:
#| eval: false
using Pkg
Pkg.develop(path = "~/Documents/EcologicalNetworksDynamics.jl-main/")

In [ ]:
#| eval: false
## My first BEFW Modelling

## Packages I need
using BEFWM2, DataFrames, Random, Plots, EcologicalNetworksPlots

## Time to do some Experiments!

### Preamble

One of main advantages of running food web models in Julia is that simulations are fast and can be readily stored in your active project. With this in mind, make a new folder in your project called `out_objects` (right click > New Folder).

### A first run of the Ecological Network Dynamics model (BEFW)

There are four major steps when running the BioEnergetic Food Web model in Julia.  These should be familiar from our introduction to the `DifferentialEquations` package:

1. Generate an initial food web network
2. Set the parameters for each species in the network to generate the equations
3. Simulate the network and equations
4. Explore output and plot

While in the previous example with `Differential Equations` we assumed a simple 2-species network, one of the new activities here is to take advantage of a rich history of theory and tools to construct species rich networks with appropriate structural properties, such as _connectance/complexity_ and levels of _generalism/specialism_ and things the number of trophic levels and a body size distribtion of the species across trophic levels.

#### Step 1: generate an initial network

Here we make a foodweb, actually, a food chain, from an _adjacency matrix_ with the FoodWeb method.


In [ ]:
A = [0 0 0; 1 0 0; 0 1 0] # 1 basal producer ⋅ Species 2 eats 1 ⋅ Species 3 eats 2
foodweb = FoodWeb(A)

#### Step 2: Generate the model parameters

Once the foodweb is created, the next step is to attribute values to the model parameters. This can be simply done by calling the method ModelParameters with foodweb as an argument.


In [ ]:
# construct the equations and fixed parameters
# see below for body size dependent parameters etc
params = ModelParameters(foodweb)

#### Step 3: Simulate biomass dynamics

Everything is ready to run the simulation, which can be simply done by calling simulate with the model parameters (params) and a vector species' initial biomass (B0).


In [ ]:
# create body sizes for each species
B0 = [0.5, 0.5, 0.5]

# use simulate function
# builds equations and uses DiffEq to run them!
sim = simulate(params, B0)

#### Step 4: Seeing the outputs!

Eventually you may want to plot the biomass dynamics - the trajectory -  of your community to see what is happening. For our minimal example, it can be done as follows:


In [ ]:
# create multiple objects: time = t and Bx = biomass for each species
# note how julia allows multiple things on left of = sign!
t, B1, B2, B3 = sim.t, sim.B[:,1], sim.B[:,2], sim.B[:,3]; # unpack variables

# Plot the basal species
plot(t, B1, lw = 3, label="Producer", xlabel = "Time", ylabel = "Biomass")
# add the herbivore
plot!(t, B2, lw = 3, label="Consumer")
# add the top predator
plot!(t, B3, lw = 3, label="Top consumer")

## A More Complex Example

#### Step 1: Generae the initial network

In order to run the BEFW model with a more complex network, we have to construct an initial food web network (an adjacency matrix) using [the niche model](https://www.nature.com/articles/35004572?cacheBust=1510239451067). The network is characterised by the number of species in the network and its [connectance/complexity](https://en.wikipedia.org/wiki/Ecological_network) value.

Note that we are now using functionality provided by the `EcologicalNetworks` package.


In [ ]:
S = 20; # define the number of species
C = 0.2; # define the connectance (complexity) of the network

# construct the food web
foodweb_niche = FoodWeb(nichemodel, S, C=C)

# see it:
foodweb_niche.A

As above, our next step is to define a vector of bodymasses and then pass this, and the network to the `simulate` function.  Here we combine the `Uniform` function from the _Distributions_ package with the `rand` function from the _Random_ package.


In [ ]:
# construct the equations and fixed parameters
# see below for body size dependent parameters etc
params_niche = ModelParameters(foodweb_niche)

# define bodymasses between 0 and 1 and get S = 20 of them.
# this will ensure your plot looks like the one in the document
Random.seed!(123)
B0 = rand(S)

# simulate using params and bodymasses
sim_niche = simulate(params_niche, B0, stop = 100)

Now we can move to plotting again.  Note how we now ask for the time directly from the simulate object and all of the biomasses from that object as well.

Note too how we can supress the legend (which covers some of the time series).


In [ ]:
plot(sim_niche[:t], (sim_niche[:B]), legend = false)

One game to play now is to alter the bodymass distribution. `rand` selects a randon uniform number between 0 and 1.  Can you figure out how to make the distribution uniform between 0 and 10?  See what that does.